In [1]:
import textwrap
import csv
import json
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import time
import google.generativeai as genai
import re 

from IPython.display import display
from IPython.display import Markdown
import ast 
import pandas as pd
from collections import Counter
import numpy as np
from scipy.spatial import distance
import os
from pathlib import Path
from sklearn.metrics import f1_score
import eval_metrics as em
from sklearn.metrics import recall_score, balanced_accuracy_score

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY= ''
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')

RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

In [2]:
sample = pd.read_csv("./goemotion_data/sample.csv")
example = pd.read_csv("./goemotion_data/example.csv")
example_dict = json.load(open("./goemotion_data/examples_dict.json"))
unique_labels = ["neutral", "admiration", "gratitude", "approval", "amusement"]

In [9]:
gt_sample_probs = {}
for i in range(210):
    amb_labels = []
    emotion_counts = Counter(sample.loc[i,"human_label"].split(","))
    total_counts = sum(emotion_counts.values())
    probs = {emo: round(emotion_counts[emo] / total_counts, 4) for emo in unique_labels}
    gt_sample_probs[sample.loc[i,"id"]] = [probs[emo] for emo in unique_labels]

In [126]:
def example_sampling(example, example_dict, num):
    multi_num = num//3 + 1
    single_num = num - multi_num

    multi_df = example[example['type'] == 2]
    single_df = example[example['type'] == 1]

    multi_index = multi_df.sample(multi_num,  random_state=10, weights='weights')
    single_index = single_df.sample(single_num,  random_state=10, weights = 'weights')

    cur_example_df = pd.concat([multi_index, single_index]).reset_index()

    example_text = "Example: \n" + "\n".join(f"Example {i+1}: {cur_example_df.loc[i,'text']}. Emotion probabilities: {example_dict[cur_example_df.loc[i, 'id']]['emo_dict']}" for i in list(cur_example_df.index))
    return example_text, cur_example_df

In [162]:
example_text, cur_example_df = example_sampling(example, example_dict, 15)

In [164]:
SYSTEM_PROMPT_GENERATE_GOEMOTION = ' '.join([
    "You are an emotionally-intelligent and empathetic agent.",
    "You will be given a piece of text and some examples, and your task is to predict the probability of the emotion of a target text. "
])
RULES = " ".join([
    "Choose the emotions from [neutral, admiration, gratitude, approval, amusement]. Output statisfies the following rules\n",
    "Rule 1: Generate a dictionary of emotion probabilities in format of {'neutral': 0.1, 'admiration':0.0, 'gratitude':0.1, 'approval':0.8, 'amusement':0.0}.",
    "Rule 2: Ensure the sum of probability equal to 1.\n",
    "Rule 3: Do not explain, only the dictionary.\n",
    "Please check again whether your output follows the three rules."
])
USER_PROMPT_RETRY_INSTRUCTION = "Please only pick from the given options separated by comma."


In [165]:
def gemini_pred_emotion(text, example_text):
    time.sleep(0.2)
    prompt = SYSTEM_PROMPT_GENERATE_GOEMOTION + example_text + f"The target text is :{text}." + RULES
    response = model.generate_content(prompt)
    return response, prompt

In [168]:
def identify_format(text):
    match = re.search(r"\{.*\}", text)
    if match:
        text = match.group(0)
    result_dict = ast.literal_eval(text)
    return result_dict
def dictToList(dict, unique_labels):
    prob_list = [dict[emo] for emo in unique_labels]
    return prob_list

In [169]:
def batch10(turn, example_text):
    pred_emo = {}
    log = {}
    for i in range(10):
        text = sample.loc[turn*10+i, 'text']
        ID = sample.loc[turn*10+i, 'id']
        try:
            response, prompt = gemini_pred_emotion(text, example_text)
            response = response.text.strip()
            response = identify_format(response)
            pred_emo[ID] = dictToList(response, unique_labels)
            log[ID] = [prompt, text, response, sample.loc[turn*10+i, 'human_label']]
        except:
            try: 
                time.sleep(60)
                response, prompt = gemini_pred_emotion(text, example_text)
                response = response.text.strip()
                
                response = identify_format(response)
                pred_emo[ID] = dictToList(response, unique_labels)

                log[ID] = [prompt, text, response,sample.loc[turn*10+i, 'human_label']]
            except:
                print(ID,'Gemini api has an error.: ', text)
                log[ID] = [text, prompt, sample.loc[turn*10+i, 'human_label']]
                pred_emo[ID] = [1,0,0,0,0]
    return pred_emo, log

In [170]:
pred_emo_t = {}
log_t = {}
for turn in range(0,21):
    print("Current turn:", turn)
    time.sleep(turn)
    pred_emo, log = batch10(turn, example_text)
    pred_emo_t.update(pred_emo)
    log_t.update(log)

Current turn: 0
Current turn: 1
Current turn: 2
Current turn: 3
edg883j Gemini api has an error.:  Why yes that ass is flawless
Current turn: 4
Current turn: 5
Current turn: 6
Current turn: 7
Current turn: 8
Current turn: 9
Current turn: 10
Current turn: 11
Current turn: 12
Current turn: 13
Current turn: 14
Current turn: 15
Current turn: 16
Current turn: 17
Current turn: 18
Current turn: 19
Current turn: 20
